<a href="https://colab.research.google.com/github/Maede27/LSTM-Recurrent-Neural-Network-with-Keras/blob/master/COVID_19_Tweet_Classification_with_LSTM_Recurrent_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import time
from tensorflow import keras

## Load the train and test tweet datasets

In [2]:
from google.colab import files
train_data = files.upload()

Saving tweet_train.csv to tweet_train.csv


In [3]:
test_data = files.upload()

Saving tweet_test.csv to tweet_test.csv


In [4]:
type(train_data)

dict

In [5]:
# change the type of data to Pandas Dataframe
import io
train_data = pd.read_csv(io.BytesIO(train_data['tweet_train.csv']))


In [6]:
type(train_data)

pandas.core.frame.DataFrame

In [7]:
test_data = pd.read_csv(io.BytesIO(test_data['tweet_test.csv']))
type(test_data)


pandas.core.frame.DataFrame

In [8]:
train_data.head().T

,0,1,2,3,4
text,If I smelled the scent of hand sanitizers toda...,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,@diane3443 @wdunlap @realDonaldTrump Trump nev...,@brookbanktv The one gift #COVID19 has give me...,25 July : Media Bulletin on Novel #CoronaVirus...
token,if i smelled the scent of hand sanitizers toda...,hey @yankees @yankeespr and @mlb - wouldn't it...,@diane3443 @wdunlap @realdonaldtrump trump nev...,@brookbanktv the one gift #covid19 has give me...,25 july : media bulletin on novel #coronavirus...
emoji,False,False,False,False,False
sentiment,Positive,Negative,Negative,Positive,Positive


In [9]:
cleanup_nums = {"sentiment": {"Positive": 2, "Neutral": 1, "Negative": 0}}
train_data.replace(cleanup_nums, inplace=True)
test_data.replace(cleanup_nums, inplace=True)

## Remove punctuations, html tags, emojis, URLs and stopwords


In [10]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_word= set(stopwords.words("english"))

def remove_stopwords(text):
  text= [word.lower for word in text.split() if word.lower() not in stop_word]
  return " ".join(text)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
import re

def remove_url(text):
  clean=re.compile(r"(https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*")
  return clean.sub(r"", text)

In [12]:
def remove_html_tags(text):
  clean=re.compile(r"<.*?>")
  return clean.sub(r"", text)

In [13]:
def remove_emojis(string):
  clean= re.compile(
          u"\U0001F600-\U0001F64F"  # emoticons
          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
          u"\U0001F680-\U0001F6FF"  # transport & map symbols
          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
          u"\U00002702-\U000027B0"
          u"\U000024C2-\U0001F251"
          u"\U0001f926-\U0001f937"
          u"\u200d"
          u"\u2640-\u2642" 
          "]+", flags=re.UNICODE
  )
  return clean.sub(r"", string)
                      


In [14]:
import string

def remove_punctuations(text):
  translator = str.maketrans("", "", string.punctuation)
  return text.translate(translator)

In [15]:
train_data["text"] = train_data.text.map(lambda x: remove_url(x))
train_data["text"] = train_data.text.map(lambda x: remove_html_tags (x))
train_data["text"] = train_data.text.map(lambda x: remove_emojis (x))
train_data["text"] = train_data.text.map(lambda x: remove_punctuations(x))
train_data["text"]

0        If I smelled the scent of hand sanitizers toda...
1        Hey Yankees YankeesPR and MLB  wouldnt it have...
2        diane3443 wdunlap realDonaldTrump Trump never ...
3        brookbanktv The one gift COVID19 has give me i...
4        25 July  Media Bulletin on Novel CoronaVirusUp...
                               ...                        
14195    You do NOT need to shave your beard to protect...
14196    Distribution of Groceries amp vegetable Kit to...
14197    His own racism cost Trump the chance to be a h...
14198    1 Helpful 2 Relevant 3 Contextual  stories nee...
14199    As a doctor and asthmatic I can tell you if yo...
Name: text, Length: 14200, dtype: object

In [16]:
from collections import Counter

def word_count(text): 
  count= Counter()
  for i in text.values:
    for word in i.split():
      count[word]=count[word]+1
  return count

In [17]:
text = train_data["text"]
countWords= word_count(text)

number_of_words= len(countWords)
number_of_words


37012

In [18]:
countWords

Counter({'If': 301,
         'I': 1082,
         'smelled': 2,
         'the': 6851,
         'scent': 1,
         'of': 4556,
         'hand': 34,
         'sanitizers': 3,
         'today': 256,
         'on': 1666,
         'someone': 61,
         'in': 3896,
         'past': 52,
         'would': 162,
         'think': 124,
         'they': 325,
         'were': 207,
         'so': 301,
         'intoxicated': 1,
         'that…': 24,
         'Hey': 39,
         'Yankees': 4,
         'YankeesPR': 1,
         'and': 3332,
         'MLB': 11,
         'wouldnt': 7,
         'it': 725,
         'have': 1031,
         'made': 81,
         'more': 519,
         'sense': 19,
         'to': 5543,
         'players': 9,
         'pay': 48,
         'their': 301,
         'respects': 1,
         'A…': 18,
         'diane3443': 1,
         'wdunlap': 1,
         'realDonaldTrump': 231,
         'Trump': 159,
         'never': 61,
         'once': 32,
         'claimed': 8,
         'COVID1

In [21]:
# Maximum numbers of words in a sequence
seq_max_len= 30

In [22]:
train_data.dtypes

text          object
token         object
emoji         object
sentiment    float64
dtype: object

## split data into train and test subsets

In [23]:
split = int(train_data.shape[0] * 0.75)
train_sentence = train_data.text[:split]
train_label= train_data.sentiment[:split]

test_sentence = train_data.text[split:]
test_label= train_data.sentiment[split:]


In [24]:
from keras.preprocessing.text import Tokenizer
token= Tokenizer(num_words = number_of_words)
token.fit_on_texts(train_sentence)

In [25]:
word_index= token.word_index

In [26]:
word_index

{'covid19': 1,
 'the': 2,
 'to': 3,
 'of': 4,
 'in': 5,
 'and': 6,
 'a': 7,
 'for': 8,
 'is': 9,
 'on': 10,
 'cases': 11,
 'you': 12,
 'are': 13,
 'with': 14,
 'this': 15,
 'coronavirus': 16,
 'new': 17,
 'at': 18,
 'from': 19,
 'have': 20,
 'we': 21,
 'that': 22,
 'i': 23,
 'as': 24,
 'has': 25,
 'your': 26,
 'be': 27,
 'it': 28,
 'our': 29,
 'by': 30,
 'not': 31,
 'amp': 32,
 'if': 33,
 'people': 34,
 'will': 35,
 'positive': 36,
 'more': 37,
 'all': 38,
 'how': 39,
 'us': 40,
 'about': 41,
 'pandemic': 42,
 'but': 43,
 'can': 44,
 'who': 45,
 'my': 46,
 'deaths': 47,
 'been': 48,
 'now': 49,
 'so': 50,
 'out': 51,
 'an': 52,
 'health': 53,
 'what': 54,
 'or': 55,
 'they': 56,
 'total': 57,
 'was': 58,
 'help': 59,
 'get': 60,
 'no': 61,
 'one': 62,
 'its': 63,
 'during': 64,
 'up': 65,
 'july': 66,
 'just': 67,
 'today': 68,
 'covid19…': 69,
 'do': 70,
 'mask': 71,
 'he': 72,
 'their': 73,
 'covid': 74,
 'when': 75,
 'daily': 76,
 'after': 77,
 'spread': 78,
 'like': 79,
 'even': 80

In [27]:
# Convert all rows in your corpus to string object in order not to get the error : AttributeError: 'int' object has no attribute 'lower'
# train_sentence = [str (item) for item in train_sentence]
# train_sentence = [item for item in train_sentence if not isinstance(item, int)] 

In [28]:
train_seq = token.texts_to_sequences(train_sentence)
train_seq[0]

[33,
 23,
 6211,
 2,
 9683,
 4,
 569,
 4705,
 68,
 10,
 345,
 5,
 2,
 531,
 23,
 146,
 205,
 56,
 108,
 50,
 9684,
 1218]

## Pad the sequences to have a specific length (seq_max_len= 40)

In [29]:
from keras.preprocessing.sequence import pad_sequences

#padding for train data:
train_pad= pad_sequences (train_seq, maxlen= seq_max_len, padding= "post" , truncating= "post")


In [30]:
train_pad[0]

array([  33,   23, 6211,    2, 9683,    4,  569, 4705,   68,   10,  345,
          5,    2,  531,   23,  146,  205,   56,  108,   50, 9684, 1218,
          0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [31]:
#padding for test data:
# test_sentence = [str (item) for item in test_sentence] 
test_seq = token.texts_to_sequences(test_sentence)

In [32]:
test_pad= pad_sequences (test_seq, maxlen= seq_max_len, padding= "post" , truncating= "post")

In [33]:
test_pad[0]

array([  118,   205,     2,    40,     9, 10419,   147,    14,  5473,
           1,     5,    54,    52,   269,   166,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0], dtype=int32)

In [34]:
print(train_data.text[0])
print(train_seq[0])

If I smelled the scent of hand sanitizers today on someone in the past I would think they were so intoxicated that… 
[33, 23, 6211, 2, 9683, 4, 569, 4705, 68, 10, 345, 5, 2, 531, 23, 146, 205, 56, 108, 50, 9684, 1218]


### Check inverse

In [35]:
reversed_index= dict ([(value, key) for (key, value) in word_index.items()])

In [36]:
def decode(text):
  return " ".join([reversed_index.get(i, "?") for i in text])

In [37]:
decode(train_seq[0])

'if i smelled the scent of hand sanitizers today on someone in the past i would think they were so intoxicated that…'

In [38]:
print(f"shape of train{train_pad.shape}")

shape of train(10650, 30)


In [39]:
print(f"shape of test{test_pad.shape}")

shape of test(3550, 30)


In [40]:
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dropout, Dense
from keras.initializers import Constant
from keras.optimizers import Adam

In [41]:
RNN_model = Sequential() 

RNN_model.add(Embedding(number_of_words, 32, input_length= seq_max_len))
RNN_model.add(LSTM(64, dropout=0.1))
RNN_model.add(Dense(1, activation= "sigmoid"))


In [42]:
optimizer= Adam(learning_rate= 3e-4)
RNN_model.compile( loss="binary_crossentropy", optimizer= optimizer, metrics= ["accuracy"])

In [43]:
RNN_model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 32)            1184384   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                24832     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 1,209,281
Trainable params: 1,209,281
Non-trainable params: 0
_________________________________________________________________


In [44]:
history = RNN_model.fit(
    train_pad, train_label, epochs= 20, validation_data=(test_pad, test_label),
)

Epoch 1/20
333/333 [==============================] - 8s 24ms/step - loss: -0.4943 - accuracy: 0.3830 - val_loss: nan - val_accuracy: 0.3972
Epoch 2/20
333/333 [==============================] - 8s 23ms/step - loss: -3.7386 - accuracy: 0.5186 - val_loss: nan - val_accuracy: 0.4837
Epoch 3/20
333/333 [==============================] - 8s 23ms/step - loss: -6.6530 - accuracy: 0.5898 - val_loss: nan - val_accuracy: 0.5062
Epoch 4/20
333/333 [==============================] - 8s 23ms/step - loss: -9.3482 - accuracy: 0.6230 - val_loss: nan - val_accuracy: 0.4941
Epoch 5/20
333/333 [==============================] - 7s 22ms/step - loss: -11.6939 - accuracy: 0.6384 - val_loss: nan - val_accuracy: 0.5068
Epoch 6/20
333/333 [==============================] - 8s 23ms/step - loss: -13.8102 - accuracy: 0.6474 - val_loss: nan - val_accuracy: 0.5141
Epoch 7/20
333/333 [==============================] - 7s 22ms/step - loss: -15.4369 - accuracy: 0.6372 - val_loss: nan - val_accuracy: 0.5051
Epoch 8/20